In [1]:
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from lxml import etree
import os
from os import path
import shutil
import time
import demjson
from selenium.webdriver.chrome.options import Options
from multiprocessing.dummy import Pool

In [2]:
def getResponse(url):
    headers = {
        'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36'
    }
    response = requests.get(url=url, headers=headers)
    return response

In [3]:
def getHTML(url):
    response = getResponse(url)
    response.encoding = 'utf-8'
    return response.text

In [4]:
# 通过selenium获取page_text
def getHTMLBySelenium(url):
    chrome_options =Options()
    chrome_options.add_argument('--headless')
    browser = webdriver.Chrome(options=chrome_options)
    #browser = webdriver.Chrome()
    browser.get(url)
    time.sleep(2)
    page_text = browser.page_source
    browser.quit()
    return page_text

In [37]:
print(getHTMLBySelenium('https://fm.qq.com/album/rd002M7d631Huzk3'))

<html lang="zh-cn" class="os_win7"><head>
<meta charset="utf-8">
<meta http-equiv="Content-Type" content="text/html; charset=utf-8">
	<!--wnsdiff-base-->	<!--wnsdiff-base-end-->
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta name="x5-pagetype" content="webapp">
<title>投资财经频道__企鹅FM</title><meta itemprop="description" name="description" content="通俗易懂的投资理财课。">
<meta name="keywords" content="投资财经频道，qq,喜马拉雅听书,网易云音乐,鬼吹灯电子书,虾米播放器,蜻蜓收音机,凤凰新闻,晋江免费小说,荔枝,航旅纵横,陌秀,快手,得到,小米运动,单车,yy语音,唱吧录音K歌,酷我,樊登读书会,echo,豆瓣"><meta name="applicable-device" content="pc">	
<link rel="Shortcut Icon" href="https://qzonestyle.gtimg.cn/qz-proj/fm-official-website/img/favicon.ico?max_age=31536000" type="image/x-icon">

<script>//interface里面这个变量
window.g_domain = document.domain;
window._s_ = new Date();	window.g_isHttps = true;</script>
<script>
window.g_qzonetoken = window.shine0callback = '';
</script>
<script>
(function(){	if(window!=top){
		top.location.href = window.location.href;
		return;
	}	var version 

In [5]:
def getSrcUrl(url):
    page_text = getHTMLBySelenium(url)
    tree = etree.HTML(page_text)
    src = tree.xpath('//audio/@src')
    if len(src) == 0:
        return ""
    return src[0]

In [6]:
def getPageUrl(url):
    page_text = getHTMLBySelenium(url)
    tree = etree.HTML(page_text)
    all_li = tree.xpath('//ul[@class="sound-list"]/li')
    linkList = []
    for li in all_li:
        # https://fm.qq.com/show/rd002Rf5g00asGbb__
        pageUrl = li.xpath('./a/@href')
        #print(pageUrl)
        if len(pageUrl) == 0:
            continue
        linkList.append(pageUrl[0])
    return linkList

In [7]:
for temp in getPageUrl('https://fm.qq.com/album/rd002M7d631Huzk3'):
    print(temp)

https://fm.qq.com/show/rd003bvzU43LQ3DY__
https://fm.qq.com/show/rd003Akrf54G6WQc__
https://fm.qq.com/show/rd000YC2Be4RAfUs__
https://fm.qq.com/show/rd00146sTf2nxMMY__
https://fm.qq.com/show/rd004PBIHb4IrW3J__
https://fm.qq.com/show/rd001P1hph1xfhSF__
https://fm.qq.com/show/rd001Ny4iC2tSpNB__
https://fm.qq.com/show/rd000wcxQf43BsFX__
https://fm.qq.com/show/rd003uUwwb3BZkBZ__
https://fm.qq.com/show/rd00495UGv2LaQHm__


In [ ]:
# https://fm.qq.com/webapp/json/luobo_discover/GetAlbumListByCategory?g_tk=5381&type=8&pageType=0&commonInfo.cookie=offset%3D18&commonInfo.isRefresh=0&commonInfo.hasMore=1&commonInfo.noUpdate=0&commonInfo.debugModeInfo=&commonInfo.messageLevel=0&commonInfo.message=&commonInfo.appCookies=&categoryId=39129&inCharset=utf-8&outCharset=utf-8&_=1618467113565
# https://fm.qq.com/webapp/json/luobo_discover/GetAlbumListByCategory?g_tk=5381&type=8&pageType=0&commonInfo.cookie=offset%3D90&commonInfo.isRefresh=0&commonInfo.hasMore=1&commonInfo.noUpdate=0&commonInfo.debugModeInfo=&commonInfo.messageLevel=0&commonInfo.message=&commonInfo.appCookies=&categoryId=39129&inCharset=utf-8&outCharset=utf-8&_=1618467113569
# https://fm.qq.com/webapp/json/luobo_discover/GetAlbumListByCategory?g_tk=5381&type=8&pageType=0&commonInfo.cookie=offset%3D216&commonInfo.isRefresh=0&commonInfo.hasMore=1&commonInfo.noUpdate=0&commonInfo.debugModeInfo=&commonInfo.messageLevel=0&commonInfo.message=&commonInfo.appCookies=&categoryId=39129&inCharset=utf-8&outCharset=utf-8&_=1618467004510
# https://fm.qq.com/webapp/json/luobo_discover/GetAlbumListByCategory?g_tk=5381&type=8&pageType=0&commonInfo.cookie=offset%3D252&commonInfo.isRefresh=0&commonInfo.hasMore=1&commonInfo.noUpdate=0&commonInfo.debugModeInfo=&commonInfo.messageLevel=0&commonInfo.message=&commonInfo.appCookies=&categoryId=39129&inCharset=utf-8&outCharset=utf-8&_=1618467004512

In [24]:
# https://fm.qq.com/category/39126_39129
url = 'https://fm.qq.com/category/39126_39129'
page_text = getHTMLBySelenium(url)
tree = etree.HTML(page_text)
all_li = tree.xpath('//ul/li')
print(len(all_li))

19


In [26]:
for li in all_li:
    albumUrl = 'https:' + li.xpath('./div/a/@href')[0]
    print(albumUrl)

https://fm.qq.com/album/rd001R8Ehl21WZNB
https://fm.qq.com/album/rd0032DyR64CMpFK
https://fm.qq.com/album/rd001nfIki1idO1h
https://fm.qq.com/album/rd004brmua4BkMng
https://fm.qq.com/album/rd003UQHoy4ANt7S
https://fm.qq.com/album/rd0007Q6NV1UACJ6
https://fm.qq.com/album/rd002TWmdq4exp4m
https://fm.qq.com/album/rd002D52uU0ggzIl
https://fm.qq.com/album/rd000wBesb1UESk1
https://fm.qq.com/album/rd000u9UGn1fOGRF
https://fm.qq.com/album/rd001a56yh2mLbeB
https://fm.qq.com/album/rd001LevQA1BuTgC
https://fm.qq.com/album/rd003wb4jv06mtbh
https://fm.qq.com/album/rd002XbFl101X5FQ
https://fm.qq.com/album/rd003e1U6J3wq1s6
https://fm.qq.com/album/rd0025nWoj1rzQn3
https://fm.qq.com/album/rd003aCaBZ4b2wD8
https://fm.qq.com/album/rd001aRU9E2Y7wqK
https://fm.qq.com/album/rd000GQjei2enrPU


In [8]:
with open('albumUrl.txt', 'r', encoding='utf-8') as fp:
    data = fp.read()

In [9]:
# <a href="//fm.qq.com/album/rd004RxHY92RoaIr" class="cover-mask" rel="nofollow">

In [10]:
ex = '<a href="//fm.qq.com/album(.*?)" class="cover-mask" '
allAlbumUrl = re.findall(ex, data, re.M)

In [11]:
print(len(allAlbumUrl))

57


In [12]:
linkList = []
for temp in allAlbumUrl:
    print(temp)
    linkList.append('https://fm.qq.com/album' + temp)

/rd0037lDmY0CTWTA
/rd004brmua4BkMng
/rd0007Q6NV1UACJ6
/rd002TWmdq4exp4m
/rd0037ouEk1ubpwq
/rd002yrkDr4YyJg5
/rd003VJELH1WbJMO
/rd004BSmPB1XyUXS
/rd0019KTlX2DqQzo
/rd000d3z1L00O3re
/rd001plW1Y2qyXYf
/rd001mWMMb3C8WJ9
/rd000ajvuc2EtwFS
/rd002M7d631Huzk3
/rd001LdWdb1tlwM8
/rd001i948h3fSl3P
/rd001zC0HL0bQZnu
/rd004eE5Rg4WH46E
/rd002Txc5B46eZfM
/rd002ryXoh3GwkJY
/rd002haTxI35Wb4G
/rd000u9UGn1fOGRF
/rd002kqdUQ1T0X86
/rd004VJntB34cEvQ
/rd004FGLAT2ZDgtR
/rd0019T2AA1ais5U
/rd000pGi6S19RKg8
/rd001uglyI2YvPUl
/rd003k4viD0pCOim
/rd004BRpsv36TObx
/rd003ugrpX4QVkxM
/rd002D52uU0ggzIl
/rd002yrkDr4YyJg5
/rd0037ouEk1ubpwq
/rd003BFr7v4MjAgw
/rd003VJELH1WbJMO
/rd004BSmPB1XyUXS
/rd0038S0zR0DDSpc
/rd004TLpAu4I5zxr
/rd0011aeEY3n4Vwl
/rd003hqTMV0iU57Z
/rd002sfFfV2rns3u
/rd003WLO8C1rERBg
/rd002ywN2w40LVPs
/rd003OnxWt0GrYKP
/rd001mkLxp35bdhB
/rd002QzJr60r0geF
/rd003msYB04deyCa
/rd004GhwRL2FLVTi
/rd004g5Rte15oyg1
/rd001qBmgJ2z12rY
/rd001QJDz22r5hF0
/rd000f5d9W0maygX
/rd003xhgPZ1SY7pW
/rd0012aWyY08IEil
/rd00321hL

In [14]:
for link in linkList:
    print(link)

https://fm.qq.com/album/rd0037lDmY0CTWTA
https://fm.qq.com/album/rd004brmua4BkMng
https://fm.qq.com/album/rd0007Q6NV1UACJ6
https://fm.qq.com/album/rd002TWmdq4exp4m
https://fm.qq.com/album/rd0037ouEk1ubpwq
https://fm.qq.com/album/rd002yrkDr4YyJg5
https://fm.qq.com/album/rd003VJELH1WbJMO
https://fm.qq.com/album/rd004BSmPB1XyUXS
https://fm.qq.com/album/rd0019KTlX2DqQzo
https://fm.qq.com/album/rd000d3z1L00O3re
https://fm.qq.com/album/rd001plW1Y2qyXYf
https://fm.qq.com/album/rd001mWMMb3C8WJ9
https://fm.qq.com/album/rd000ajvuc2EtwFS
https://fm.qq.com/album/rd002M7d631Huzk3
https://fm.qq.com/album/rd001LdWdb1tlwM8
https://fm.qq.com/album/rd001i948h3fSl3P
https://fm.qq.com/album/rd001zC0HL0bQZnu
https://fm.qq.com/album/rd004eE5Rg4WH46E
https://fm.qq.com/album/rd002Txc5B46eZfM
https://fm.qq.com/album/rd002ryXoh3GwkJY
https://fm.qq.com/album/rd002haTxI35Wb4G
https://fm.qq.com/album/rd000u9UGn1fOGRF
https://fm.qq.com/album/rd002kqdUQ1T0X86
https://fm.qq.com/album/rd004VJntB34cEvQ
https://fm.qq.co

In [15]:
linkList = list(set(linkList))

In [16]:
print(len(linkList))

53


In [17]:
pageUrls = []
srcUrls = []
for link in linkList:
    print(link)
    for pageUrl in getPageUrl(link):
        print(pageUrl)
        pageUrls.append(pageUrl)
        srcUrl = getSrcUrl(pageUrl)
        print(srcUrl)
        srcUrls.append(srcUrl)        

https://fm.qq.com/album/rd001qBmgJ2z12rY
https://fm.qq.com/show/rd003f5zZi2aCQI0__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003f5zZi2aCQI0.m4a?fromtag=36&vkey=90BF8C911FB5C80E90BA97BC0F763C131A304243D0CA6C910C6E503A97BA36CB3D5944F5F300D15B47ACC6469B04C8FD2B6E58BF1ADE5892&guid=10000
https://fm.qq.com/show/rd000Fkv772fkp4T__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196000Fkv772fkp4T.m4a?fromtag=36&vkey=B68879FF832E05B3D72523276C6C561D1694D8CFD17870D5E7A87753C9646ABC55E38FA385AA3C1FB4D5A76EBF0D8F71BC18A28C0D54FAD6&guid=10000
https://fm.qq.com/show/rd002Keqwb1VHSuJ__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196002Keqwb1VHSuJ.m4a?fromtag=36&vkey=6855D458AD4B7057C2A42FD685FE0984BD390899DA616E3E1EDB88DD7E76F6D9D74EADD3CCD960906500EF9F5DD272DF4A1492C9728593D5&guid=10000
https://fm.qq.com/show/rd000cuelu1ndXJ8__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196000cuelu1ndXJ8.m4a?fromtag=36&vkey=5C0FBED0F54AFFAB24CA1DEC17BA66603993A5EC883C545ED27CB0019AAC1BAA030C18C2676D6C0C3F8B3F209CA19AFFF1F

https://fm.qq.com/show/rd0031XaUn4Sv1eh__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960031XaUn4Sv1eh.m4a?fromtag=36&vkey=18E19E13780C9CE85884C01107A5142ACD3C7A2B059AC8DFD7F79C6DC2C9C96E383DA70D5D03AD93AA32092CA5F18DB5924280F1FFA49177&guid=10000
https://fm.qq.com/show/rd004M8t6y2a6E1B__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196004M8t6y2a6E1B.m4a?fromtag=36&vkey=52CDF6657B4901E83818C11EFCBCBB5B492EE1AE87B8D0D64EB526D31264B761E6B6B98E139689FF7369080A7F072B473E85693C6E82E94B&guid=10000
https://fm.qq.com/show/rd0032oPd51FyRb8__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960032oPd51FyRb8.m4a?fromtag=36&vkey=315A9DB7C3DDD8F4ECBBB1D9D7D17DEFCAECCA583E07A18D53D202D92CEBDCCB75349FC3E92B5D4FEDE28635C0D90926C3013958D2373E9B&guid=10000
https://fm.qq.com/show/rd000CDRuK1Yg24b__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196000CDRuK1Yg24b.m4a?fromtag=36&vkey=A5CA3D4472628F275C3370E3CE0F6E248E0679594B984EF478E2D91FDED730F2B4A369B906B08BD954FEB5A62FD7B73F4A0F6F464166BEFF&guid=10000
https://fm.qq.co

http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196004FzNQo3Lp1KX.m4a?fromtag=36&vkey=EFA4D0BB7F5E5C9CE58E2A79F48F37F0771BC3431A8366F18669414BA6F8374DA222D2F641C5C4E93F7CA883A3C486ACF174CD70765667E4&guid=10000
https://fm.qq.com/show/rd0049EW2V2yuTIl__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960049EW2V2yuTIl.m4a?fromtag=36&vkey=2C38B7FB50ADA8205F1EFE5D311EE24512BA2FAA1026717C46DAB53705AD0F83F1993CB542ABA1F70A86D4C8FCEB225B238E2EFAF1A4D38B&guid=10000
https://fm.qq.com/show/rd001Sv1XZ0ckKfH__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196001Sv1XZ0ckKfH.m4a?fromtag=36&vkey=3B588349CC0ED160C42B780725A80638C99F5D27A0242EBF73F4C4BAD9188AF40C1D37C8E3FA66146646BC1687FE66AFAAE6313A51923DBD&guid=10000
https://fm.qq.com/show/rd0025GPsG1NHOlU__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960025GPsG1NHOlU.m4a?fromtag=36&vkey=FE7BB0B452DEBB2378C4E2F68F562F3843FA273F8FE61D876C9207AF334DA63AE9F431127AA480BA16B4DC8D4CC0EF0ED88F5A67C3B77735&guid=10000
https://fm.qq.com/show/rd000TS2Tl2dbDhY__
http://ws.stream

http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003OMxZJ1os9JC.m4a?fromtag=36&vkey=CC421BC8278EB306F2B60F5F773DD42257C3D492D882155980248AD4FF17F66151B18A7BFA2B52FA19BDAAAB841F51F75E5DC6A3BC9DE536&guid=10000
https://fm.qq.com/show/rd000LZFzA2ZQuuL__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196000LZFzA2ZQuuL.m4a?fromtag=36&vkey=5981D5A046FE8BF1FE068EBD8D1E18517162CD2B2A9B915EF8F23F17EFC61B30624BE6D8435886C97138D7BE3AE144F4FA334EE30C379D45&guid=10000
https://fm.qq.com/show/rd001rIjV600XJaV__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196001rIjV600XJaV.m4a?fromtag=36&vkey=0D03674CFCAEAF2190295C3B28C1521924F5D4BEB0D9700E7F97FAFC28C1179302D20F1C7FB5176A340A5CA3EF8D30937ED3B1546905A25D&guid=10000
https://fm.qq.com/show/rd0025q7Q20ojXrL__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960025q7Q20ojXrL.m4a?fromtag=36&vkey=7215F1F52D780180F65B6BFF26D4F4EA438AA799482FDBBE9E15E3607EE4422715E23902318DB656DB0161B0FCC76736DAC8D1B57872C52C&guid=10000
https://fm.qq.com/show/rd000p8kHd41g6gn__
http://ws.stream

http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960040kxBP4ba9kT.m4a?fromtag=36&vkey=988FB9DED577A9C26FC6B5CCFBFEEBBC1C2D13982E2F3FD24E224A72238C8348C7F83A336BFCBA9C4EEA86DB3639A816E14023C31865CC7C&guid=10000
https://fm.qq.com/show/rd003WdI8i2j5oFg__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003WdI8i2j5oFg.m4a?fromtag=36&vkey=13039F0650EEBCBEFEA032AA6281CFA86F31B4318DE05CF6A60D5716CBFF456CDAC16D96A506AC4963840D995A03E038246FA0CE7CC324BC&guid=10000
https://fm.qq.com/album/rd002kqdUQ1T0X86
https://fm.qq.com/show/rd001N039f3QKMAt__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196001N039f3QKMAt.m4a?fromtag=36&vkey=E4115BCCFB4F2F51851FBF2C4671CB987C8CF584F4EAD8DBD1FE6C00088E3FA2A97C0D5739FA789C81718FB6FE49A871CF6C48006BB4495B&guid=10000
https://fm.qq.com/show/rd003WWipf3I2Tw7__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003WWipf3I2Tw7.m4a?fromtag=36&vkey=C9A1A2BD1A13CC402521394A862673D8D4A12F4E05C9B5E43362F9E76F21EDB532322D255B8A310F36C2091468060FEDF188DB6FB6EFC73C&guid=10000
https://fm.qq.com

...

http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196001lvxEl0neZRj.m4a?fromtag=36&vkey=9B4F958A1F94FA83B722ADBC42659788B4BA1B262FBFB70BB2CECC92DB437E84470DBF5523D92F3411D1B65439F33EA84670EA4D4BDA146C&guid=10000
https://fm.qq.com/show/rd003mSTWX0norq7__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003mSTWX0norq7.m4a?fromtag=36&vkey=A835EF2B3B035CB02855FD8869072F4891459BDBE2B56E782D8F87CE30EED9A1BD454C72D67DDC9561F7A7A3D4942F3FF14ABA1A9A22096F&guid=10000
https://fm.qq.com/album/rd0037ouEk1ubpwq
https://fm.qq.com/show/rd0040FOb441CrLo__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960040FOb441CrLo.m4a?fromtag=36&vkey=52F5661045A277C1C37840A15F74B0E8D653D6212B31EDBDB924BA56780899B2F5AFA0E6CF2C67C571F00659FB28018A15714AF8BC2AB731&guid=10000
https://fm.qq.com/show/rd002AndX50rIPvX__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196002AndX50rIPvX.m4a?fromtag=36&vkey=0E21F92FED52E1D563E012C9AA474EBE4C942528E718184255B8C40C2D20D83647C98527A5A198C833F493976C7577F83E982E869DD2C346&guid=10000
https://fm.qq.com

http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196001VUZ1V1A9jwO.m4a?fromtag=36&vkey=E8790F1DFF90E50D5D1C274104B25B5BE9D2018998E754A7540012A31185726ED4D739916C251FBEA7123EB001ADFFEF9B20E16E23727A1D&guid=10000
https://fm.qq.com/show/rd0013zaGq1kavs9__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960013zaGq1kavs9.m4a?fromtag=36&vkey=419AFDC62928FDB5A2EF21066E6EA68BC45A00C8EDDE8E65EA1AA464A64003E510A8FD2B3B596082E14903C1751FD3C141ADA89FBF7F4BA9&guid=10000
https://fm.qq.com/album/rd003xhgPZ1SY7pW
https://fm.qq.com/show/rd001Yqogk1buuNU__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196001Yqogk1buuNU.m4a?fromtag=36&vkey=35ADEAF8FE48C479FD5ACB670F8CA6F3C3062754043D8FDF6F93584732D37E1ABE7B4933F2F94FB7776299F2B0D26B56ACA73637B24E377E&guid=10000
https://fm.qq.com/album/rd001uglyI2YvPUl
https://fm.qq.com/show/rd0030wRbU217kkm__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960030wRbU217kkm.m4a?fromtag=36&vkey=F35F91EFC9ACB9A46A9AFDDBA166331246883C1CE23035F30ED982B90A5E386B6E0DBDFC66F248178EEEB3D5FC13EDD12649

http://ws.stream.fm.qq.com/vfm.tc.qq.com/R1960007kkIb4DBaRs.m4a?fromtag=36&vkey=FE0A1896B476271EF3BA9CEBDAC599B80E6301FB0E0771BDE1FC274AA74088279111CA0BCB5D591D1C869DD97724230D80A783E37AF3EEC0&guid=10000
https://fm.qq.com/show/rd003hrnso0hdf8K__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003hrnso0hdf8K.m4a?fromtag=36&vkey=07812E7A360F4EE4D885852411AB85B474F12065C21C3A6BE4914704D3F65F9459A007126FAE88D4EE5B5F8CD21D41733D5F899DB484D835&guid=10000
https://fm.qq.com/show/rd003DjOsA2EwJuf__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003DjOsA2EwJuf.m4a?fromtag=36&vkey=B9412D71C4030C655EF73FF8643CBDA66771A4C0E110662A600F8FF0E3D8C06A900C30202808973A4DB98C3645F7664EBC5CCE61C23EC5E9&guid=10000
https://fm.qq.com/show/rd003q3sTY2UCcnt__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003q3sTY2UCcnt.m4a?fromtag=36&vkey=39ADA5ABDC32015592DEC9944F4EAE7B93DEA0AF9989DA311A9240374F7AA8F1BA12E264FC87C0712409D1623836E340EE61B62CB795C5DA&guid=10000
https://fm.qq.com/show/rd003npva339gcHB__
http://ws.stream

http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196002r9M0y3Jal4j.m4a?fromtag=36&vkey=8A26EBB6B579182AA53419A7BCBAFC8D613CDDC79ECC2E3FFC0E760ED583BB2C8CE26B3EFEC92291F5C39568097D790A178B9C2B918FCCD5&guid=10000
https://fm.qq.com/show/rd003podv1050zwp__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003podv1050zwp.m4a?fromtag=36&vkey=74B08D39521CD82BB8D5604B4CEF00FA8E83DDBA354331D5E249E8C68B4208E6D977BF916626373F2E762B2AD22208938E342186128C33B2&guid=10000
https://fm.qq.com/show/rd004Htz5o3McyZr__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196004Htz5o3McyZr.m4a?fromtag=36&vkey=02F765B04CF94AB44690A483944ACF3203C41A23CC999032895C386152F40BD20180C1D226E63241B75D564493CBFADC4C32CDC5DAD46A1A&guid=10000
https://fm.qq.com/show/rd004CNdBB3bYRPJ__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196004CNdBB3bYRPJ.m4a?fromtag=36&vkey=7D8A45346DEE65DD0C901FCD8B6680D4695C9D4CDAEEB6EE0E3127301993A9A598B2957A1660929F5DC62BCC5893A28FA603A3A37C411522&guid=10000
https://fm.qq.com/album/rd003WLO8C1rERBg
https://fm.qq.com

https://fm.qq.com/show/rd003k0rl13KUSMO__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003k0rl13KUSMO.m4a?fromtag=36&vkey=A9C1A4F37B4ECB1983E2FEA7769516A82346F55F1DC61FCEF6C696AEB09C0C6C7083F5E3BCFFF3829816A22CB705C2558E804C24AAC38736&guid=10000
https://fm.qq.com/album/rd000f5d9W0maygX
https://fm.qq.com/show/rd003es4Kv4SUji7__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196003es4Kv4SUji7.m4a?fromtag=36&vkey=FE2B6919D641336766E9EB40A069492D6BAE7F45DB0B5DC411EF1794AEF1E59E7D699F72B6EACDC17E788BD7D08FF079A9C358998CD14EE2&guid=10000
https://fm.qq.com/show/rd001382zk0BD8FC__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196001382zk0BD8FC.m4a?fromtag=36&vkey=BE33788D6C90AE94DFFD5B70495A6A767D8E0EF35310F995B13F5460CA04B9B10B864EEFFAD1815B1572C0B4D4DE4C634E958E51E8AF6D13&guid=10000
https://fm.qq.com/show/rd001p4Gan2KpWEc__
http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196001p4Gan2KpWEc.m4a?fromtag=36&vkey=C27B40DFAFDB6B4B7D86FADEC1A530C7335A2A8428ABD238391897CE0881C0B4A828A209E4B44AD36D1EB8B160805243FCB

In [18]:
print(len(pageUrls))
print(len(srcUrls))

385
385


In [19]:
with open('pageUrl.txt', 'w', encoding='utf-8') as fp:
    data = '\n'.join(pageUrls)
    fp.write(data)

In [20]:
with open('srcUrl.txt', 'w', encoding='utf-8') as fp:
    data = '\n'.join(srcUrls)
    fp.write(data)

In [23]:
def download(url):
    fileName = (url.rsplit('/', 1)[-1]).split('?', 1)[0]
    path = 'audio/'
    try:
        content = getResponse(url).content
        with open(path + fileName, 'wb') as fp:
            fp.write(content)
        print(fileName, 'successful')
    except Exception as e:
        print("error")

In [24]:
download('http://ws.stream.fm.qq.com/vfm.tc.qq.com/R196001B2x5s0e1PO0.m4a?fromtag=36&vkey=C9F999F4F60E86A75944AD6B5007F365477A9CCB2CB42304AAD4FDC617850D31336A1588222F75F7B13C131A6C6BE466F3537F53A76721D3&guid=10000')

R196001B2x5s0e1PO0.m4a successful


In [25]:
allObj = []
for i, url in enumerate(srcUrls):
    obj = {
        'id':i,
        'url':url
    }
    allObj.append(obj)
print(len(allObj))

385


In [26]:
def multithreadDownload(obj):
    ID = obj['id']
    url = obj['url']
    print(f'start download {ID}')
    download(url)

In [27]:
pool = Pool(4)
pool.map(multithreadDownload, allObj)
pool.close()
pool.join()

start download 0
start download 25
start download 50
start download 75
R1v9000Stg8g2QmSPE.m4a successful
start download 26
R196000R7j9q0HOPmQ.m4a successful
start download 76
R196003f5zZi2aCQI0.m4a successful
start download 1
R1v9004QzW620eATt3.m4a successful
start download 27
R196002zIf7x3Xoqni.m4a successful
start download 77
R196003180Lk1MAL2l.m4a successful
start download 51
R196000Fkv772fkp4T.m4a successful
start download 2
R1v9002VwxvX3NlZ9q.m4a successful
start download 28
R196000nIhLw3inZZc.m4a successful
start download 78
R196002Keqwb1VHSuJ.m4a successful
start download 3
R1v90007467505W1S2.m4a successful
start download 29
R196000bqOjh0XOBD8.m4a successful
start download 79
R196003eUAHJ3Gov5e.m4a successful
start download 52
R196000cuelu1ndXJ8.m4a successful
start download 4
R1v9003qMPUA2XC7Mj.m4a successful
start download 30
R196004d6mwr0Mr3JK.m4a successful
start download 80
R196004WGRpS3xw7Vv.m4a successful
start download 5
R1v9002Zqo6l0BbFKK.m4a successful
start download 3

R196000vrYBi4M6tN7.m4a successful
start download 121
R196004ZkWgs1XT9sd.m4a successful
start download 166
R196000UXo2h1gWcoU.m4a successful
start download 184
R19600409Ayc14Ik71.m4a successful
start download 122
R196002KanlF08yTRo.m4a successful
start download 167
R196001Gz1ZJ0hy8zn.m4a successful
start download 139
R196003XQ6VT3Lmh4Y.m4a successful
start download 123
R1960019p7wE3y3q5q.m4a successful
start download 168
R1960032CB093e8Lcl.m4a successful
start download 185
R196002FU2Ph1MRd1r.m4a successful
start download 124
R196002Cd8qC3VEq9p.m4a successful
start download 186
R196002V95JY0s7q8Z.m4a successful
start download 200
R196003dYWvW0kMgE6.m4a successful
start download 140
R196004BgNXM1eWLpo.m4a successful
start download 201
R19600243hmQ2TeccP.m4a successful
start download 187
R196000buB0a32EezT.m4a successful
start download 169
R196001elHzW2O7Vh3.m4a successful
start download 202
R19600168C812hSGLo.m4a successful
start download 188
R196003tDzf20PxbKu.m4a successful
start downlo

R196000jruYM2HcUQv.m4a successful
start download 350
R196001cmebA1xiANv.m4a successful
start download 327
R196003hrnso0hdf8K.m4a successful
start download 299
R196000EJPD225uZkG.m4a successful
start download 314
R296001Wu0Qu08fQtS.m4a successful
start download 351
R196003DjOsA2EwJuf.m4a successful
start download 375
R19600493e6C2Ymz0l.m4a successful
start download 328
R196004Ap41K4XDdTw.m4a successful
start download 315
R4960043jXJo1DCtXV.m4a successful
start download 352
R196003wwYw53sso1u.m4a successful
start download 329
R196003mzGHB0A3Tgq.m4a successful
start download 376
R396000peeFz4ebG4Y.m4a successful
start download 353
R196001Hee8h3pYjcv.m4a successful
start download 330
R196000bbjlC35ReVW.m4a successful
start download 316
R196001RT6Tv2UpfuR.m4a successful
start download 377
R39600255Sun2n4ChG.m4a successful
start download 354
R1960038JHAs2p7Xhf.m4a successful
start download 331
R196002K1VvB0LMG1j.m4a successful
start download 378
R1960048dqzJ0rYhTZ.m4a successful
start downlo